```
(.venv_dev311) [ec2-user@ip-10-203-107-151 bin]$ sudo yum install gcc-11*
Loaded plugins: extras_suggestions, langpacks, priorities, s3iam, update-motd
229 packages excluded due to repository priority protections
No package gcc-11* available.
Error: Nothing to do
```

# Update gcc

Resources
* https://python.langchain.com/docs/integrations/llms/llamacpp
* https://github.com/imartinez/privateGPT/issues/644
* https://amazonlinux.pkgs.org/2023/amazonlinux-x86_64/gcc-11.3.1-4.amzn2023.0.3.x86_64.rpm.html#:~:text=Download%20gcc%2D11.3.1%2D4.amzn2023.0.3.x86_64.rpm%20for%20Amazon%20Linux%202023%20from%20Amazon%20Linux%20repository.
* https://centos.pkgs.org/7/centos-sclo-rh-x86_64/devtoolset-11-gcc-c++-11.2.1-1.2.el7.x86_64.rpm.html
* https://pkgs.org/search/?q=Development%20Tools
* https://github.com/sclorg/centos-release-scl

```

#remove old gcc
sudo yum remove gcc -y
sudo yum remove gdb -y

#install scl-utils
sudo yum install scl-utils
sudo yum install centos-release-scl

#find devtoolset-11
yum list all --enablerepo='centos-sclo-rh' | grep "devtoolset"
> NOT WORKING
    yum list \*gcc\* | grep amzn2 > only shows 10-
    yum list all | grep devtoolset
    sudo yum groupinstall "Development Tools" -y
    yum list \*devtoolset\*
> Attempt
cd usr/bin/
sudo wget http://mirror.centos.org/centos/7/sclo/x86_64/rh/Packages/d/devtoolset-11-gcc-c++-11.2.1-1.2.el7.x86_64.rpm
sudo chmod 755 devtoolset-11-gcc-c++-11.2.1-1.2.el7.x86_64.rpm
sudo yum install devtoolset-11-gcc-c++-11.2.1-1.2.el7.x86_64.rpm
> Attempt2
sudo yum install centos-release-scl-rh
> Attempt3 
sudo yum-config-manager --enable rhel-server-rhscl-7-rpms
sudo yum-config-manager --add-repo=https://copr.fedoraproject.org/coprs/rhscl/centos-release-scl/repo/epel-6/rhscl-centos-release-scl-epel-6.repo
sudo yum install centos-release-scl



#install devtoolset-11-toolchain
sudo yum install devtoolset-11-toolchain -y

#add gcc 11 to PATH by adding following script to /etc/profile
PATH=$PATH::/opt/rh/devtoolset-11/root/usr/bin export PATH sudo scl enable devtoolset-11 bash

#show gcc version and gcc11 is installed successfully.
gcc --version

# FROM BOTTOM OF THREAD
CMAKE_ARGS="-DCMAKE_C_COMPILER=gcc -DCMAKE_CXX_COMPILER=g++" pip install llama-cpp-python

pip install llama-cpp-python==0.1.50```

# llama_cpp

## Resources

* https://pypi.org/project/llama-cpp-python/
* https://llama-cpp-python.readthedocs.io/en/latest/api-reference/
* https://github.com/noamgat/lm-format-enforcer/blob/main/samples/colab_llamacpppython_integration.ipynb
* https://github.com/oobabooga/text-generation-webui/issues/1534

In [2]:
import llama_cpp

ModuleNotFoundError: No module named 'llama_cpp'

In [ ]:
>>> model = llama_cpp.Llama(
...     model_path="path/to/model",
...     chat_format="llama-2",
... )

In [1]:
# Imports
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

/mnt/efs/data/AIEresearch/.venv_dev311/lib/python3.11/site-packages/torch/cuda/__init__.py:611: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [1]:

model_name_or_path = "TheBloke/Mistral-7B-Instruct-v0.2-code-ft-GPTQ"
# To use a different branch, change revision
# For example: revision="gptq-4bit-32g-actorder_True"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

prompt = "Write a story about llamas"
system_message = "You are a story writing assistant"
prompt_template=f'''/
    <|im_start|>system
    {system_message}<|im_end|>
    <|im_start|>user
    {prompt}<|im_end|>
    <|im_start|>assistant
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
print(tokenizer.decode(output[0]))

# Inference can also be done using transformers' pipeline

print("*** Pipeline:")
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    repetition_penalty=1.1
)

print(pipe(prompt_template)[0]['generated_text'])


config.json:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

PackageNotFoundError: No package metadata was found for auto-gptq